**TensorFlow Serving 安裝 **

TensorFlow Serving 可以使用 apt-get 或 Docker 安裝。在開發環境中，推薦 使用 Docker 部署 TensorFlow Serving 。不過此處出於教學目的，介紹環境依賴較少的 apt-get 安裝 。

In [ ]:
# 添加Google的TensorFlow Serving安裝來源
echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list
# 添加gpg key
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

sudo apt-get update
sudo apt-get install tensorflow-model-server

**TensorFlow Serving 模型部署**

TensorFlow Serving 可以直接讀取 SavedModel 格式的模型進行部署（匯出模型到 SavedModel 文件的方法見 前文 ）。使用以下指令即可：

tensorflow_model_server \
    --rest_api_port=埠號（如8501） \
    --model_name=模型名 \
    --model_base_path="SavedModel格式模型的資料夾絕對地址（不含版本號碼）"

**Keras Sequential 模式模型的部署** 

由於 Sequential 模式的輸入和輸出都很固定，因此這種類型的模型很容易部署，不需要其他額外操作。例如，要將 前文使用 SavedModel 匯出的 MNIST 手寫體識別模型 （使用 Keras Sequential 模式建立）以 MLP 的模型名在埠號 8501 進行部署，可以直接使用以下指令：

tensorflow_model_server \
    --rest_api_port=8501 \
    --model_name=MLP \
    --model_base_path="/home/.../.../saved"  # 資料夾絕對路徑根據自身情況填寫，無需加入版本號碼

**自定義 Keras 模型的部署**

使用繼承 tf.keras.Model 類建立的自定義 Keras 模型的自由度相對更高。因此使用 TensorFlow Serving 部署模型時，對導出的 SavedModel 文件也有更多的要求：

匯出 SavedModel 格式的方法（比如 call ）不僅需要使用 @tf.function 修飾，還要在修飾時指定 input_signature 參數，以說明輸入的形狀。該參數傳入一個由 tf.TensorSpec 組成的列表，指定每個輸入張量的形狀和類型。例如，對於 MNIST 手寫體數字辨識，我們的輸入是一個 [None, 28, 28, 1] 的四維張量（ None 表示第一維即 Batch Size 的大小不固定），此時我們可以將模型的 call 方法做以下修飾：

**在客戶端呼叫以 TensorFlow Serving 部署的模型**

TensorFlow Serving 支援以 gRPC 和 RESTful API 調用以 TensorFlow Serving 部署的模型。本手冊主要介紹較為通用的 RESTful API 方法。

RESTful API 以標準的 HTTP POST 方法進行通信，請求和回覆均為 JSON 對象。為了呼叫伺服器端的模型，我們在客戶端向伺服器發送以下格式的請求：

伺服器 URI： http://伺服器地址:埠號/v1/models/模型名:predict

**Python 客戶端範例**

以下範例使用 Python 的 Requests 庫 （你可能需要使用 pip install requests 安裝該函式庫）向本機的 TensorFlow Serving 伺服器發送 MNIST 測試集的前 10 幅圖像並返回預測結果，同時與測試集的真實標籤進行比較。

In [ ]:
import json
import numpy as np
import requests
from zh.model.utils import MNISTLoader

data_loader = MNISTLoader()
data = json.dumps({
    "instances": data_loader.test_data[0:3].tolist()
    })
headers = {"content-type": "application/json"}
json_response = requests.post(
    'http://localhost:8501/v1/models/MLP:predict',
    data=data, headers=headers)
predictions = np.array(json.loads(json_response.text)['predictions'])
print(np.argmax(predictions, axis=-1))
print(data_loader.test_label[0:10])

**Node.js 客戶端範例（Ziyang）**

以下範例使用 Node.js 將下圖轉換為 28*28 的灰階圖，發送給本機的 TensorFlow Serving 伺服器，並輸出返回的預測結果和機率。（其中使用了 圖像處理庫 jimp 和 HTTP 庫 superagent ，可使用 npm install jimp 和 npm install superagent 安裝）

In [ ]:
const Jimp = require('jimp')
const superagent = require('superagent')

const url = 'http://localhost:8501/v1/models/MLP:predict'

const getPixelGrey = (pic, x, y) => {
  const pointColor = pic.getPixelColor(x, y)
  const { r, g, b } = Jimp.intToRGBA(pointColor)
  const gray =  +(r * 0.299 + g * 0.587 + b * 0.114).toFixed(0)
  return [ gray / 255 ]
}

const getPicGreyArray = async (fileName) => {
  const pic = await Jimp.read(fileName)
  const resizedPic = pic.resize(28, 28)
  const greyArray = []
  for ( let i = 0; i< 28; i ++ ) {
    let line = []
    for (let j = 0; j < 28; j ++) {
      line.push(getPixelGrey(resizedPic, j, i))
    }
    console.log(line.map(_ => _ > 0.3 ? ' ' : '1').join(' '))
    greyArray.push(line)
  }
  return greyArray
}

const evaluatePic = async (fileName) => {
  const arr = await getPicGreyArray(fileName)
  const result = await superagent.post(url)
    .send({
      instances: [arr]
    })
  result.body.predictions.map(res => {
    const sortedRes = res.map((_, i) => [_, i])
    .sort((a, b) => b[0] - a[0])
    console.log(`我們猜這個數字是${sortedRes[0][1]}，機率是${sortedRes[0][0]}`)
  })
}

evaluatePic('test_pic_tag_5.png')